In [ ]:
# %pip install langchain_openai langchain_community python-dotenv openai langchain_core pypdf docarray

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
import os
from dotenv import load_dotenv

In [ ]:
Model = 'gpt-4o-2024-08-06'
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=Model)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
chain = model | parser

In [ ]:
from langchain.prompts import PromptTemplate
template = """
Answer to question based on the context below. If you can't answer, reply "null".
Context: {context}
Question: {question}
"""
prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is a question", question="Here is a question")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('https://www.europarl.europa.eu/doceo/document/TA-9-2024-0138_EN.pdf')
pages = loader.load_and_split()
pages

In [16]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstores = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
retriever = vectorstores.as_retriever()

/opt/homebrew/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [17]:
from operator import itemgetter
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

In [19]:
from IPython.display import Markdown
questions = [
    "What is this document about?",
    "Who is the president of US?"
]
for question in questions:
    var = chain.invoke({'question': question})
    print(var)
# Markdown(var)


The document is about a regulation aimed at improving the functioning of the internal market and promoting the uptake of human-centric and trustworthy artificial intelligence (AI), while ensuring a high level of protection of health, safety, fundamental rights, including democracy, the rule of law, and environmental protection against the harmful effects of AI systems in the Union. It includes harmonised rules for the market, prohibitions of certain AI practices, and specific requirements for high-risk AI systems.
null
